### **Ingesta y Almacenamiento de Datos**

Este notebook forma parte del trabajo final del curso de MLOps  y cubre la etapa de ingesta y almacenamiento de datos, un paso fundamental en la construcción de un pipeline de Machine Learning.

En este proceso, primero se carga un dataset de minería en formato **CSV** desde **Databricks FileStore (DBFS)** y se transforma en un **DataFrame de Apache Spark** para su procesamiento. Luego, se almacena en **formato Delta**, lo que permite un acceso más eficiente a los datos y facilita su versionado para futuras actualizaciones.

Finalmente, se valida la carga de datos consultando el contenido almacenado en **DBFS**, asegurando que la ingesta se haya realizado correctamente y que los datos estén disponibles para su uso en las siguientes etapas del pipeline de MLOps.

<style>
    p { text-align: justify; }
</style>

### 1. Inicialización del entorno en Databricks
Se inicia una sesión de Apache Spark en Databricks, lo que permitirá trabajar con grandes volúmenes de datos de manera distribuida. Luego, se define la ruta del archivo CSV con los datos de minería y se carga en un DataFrame de Spark para su procesamiento.

In [0]:
from pyspark.sql import SparkSession

# Iniciar sesión de Spark
spark = SparkSession.builder.appName("MiningMLProject").getOrCreate()

# Ruta del archivo CSV correcto
file_path = "dbfs:/FileStore/tables/Enhanced_Synthetic_Mining_Data-2.csv"

# Cargar el CSV en un DataFrame de Spark
df_spark = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar las primeras filas
df_spark.show(5)


+-----------------+------------+------------------+-----------------+-------------------------+-----------------+--------------------+--------------+----------+----+------------+-----------------+------------------+
|Accident_Occurred|Hours_Worked|Weather_Risk_Index|Machine_Age_Years|Employee_Experience_Years|Safety_Violations|Inspection_Frequency|Job_Risk_Level|Shift_Type|Area|Employee_Age|   Noise_Level_dB|     Temperature_C|
+-----------------+------------+------------------+-----------------+-------------------------+-----------------+--------------------+--------------+----------+----+------------+-----------------+------------------+
|                0|          10|0.5531357998742017|               19|                       17|                9|                  10|             1|         0|   0|          49|  63.052767719262|28.754187658419617|
|                1|          10|0.9207611469737159|                3|                        8|                5|                   1|  

### 2. Almacenamiento en formato Delta
Se almacena el DataFrame en formato Delta, un estándar optimizado para Databricks que permite versiones de los datos, consultas más rápidas y manejo eficiente de actualizaciones.


In [0]:
# Definir la ubicación en DBFS
delta_path = "dbfs:/mnt/delta/mining_data"

# Guardar el DataFrame en formato Delta
df_spark.write.format("delta").mode("overwrite").save(delta_path)

# Confirmación
print(f"Tabla guardada en: {delta_path}")


Tabla guardada en: dbfs:/mnt/delta/mining_data


### 3. Carga y validación de los datos almacenados
Para asegurarse de que los datos fueron correctamente almacenados en formato Delta, se los vuelve a cargar y se visualizan algunas filas.

In [0]:
df_spark_loaded = spark.read.format("delta").load("dbfs:/mnt/delta/mining_data")
df_spark_loaded.show(5)


+-----------------+------------+------------------+-----------------+-------------------------+-----------------+--------------------+--------------+----------+----+------------+-----------------+------------------+
|Accident_Occurred|Hours_Worked|Weather_Risk_Index|Machine_Age_Years|Employee_Experience_Years|Safety_Violations|Inspection_Frequency|Job_Risk_Level|Shift_Type|Area|Employee_Age|   Noise_Level_dB|     Temperature_C|
+-----------------+------------+------------------+-----------------+-------------------------+-----------------+--------------------+--------------+----------+----+------------+-----------------+------------------+
|                0|          10|0.5531357998742017|               19|                       17|                9|                  10|             1|         0|   0|          49|  63.052767719262|28.754187658419617|
|                1|          10|0.9207611469737159|                3|                        8|                5|                   1|  